In [1]:
%matplotlib inline

import numpy as np
from PIL import Image
from matplotlib import pyplot as plt

import tensorflow as tf

/usr/local/Cellar/python/3.7.2_2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
  return f(*args, **kwds)


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True,reshape=False)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
mnist.train.images.shape#1是通道数

(55000, 28, 28, 1)

In [14]:
with tf.Graph().as_default() as g:
    inputs=tf.placeholder(shape=[None,28,28,1],dtype=tf.float32)
    labels=tf.placeholder(shape=[None,10],dtype=tf.float32)#十分类
    
    #cnn
    #第一个卷积的权重
    conv1_weight=tf.get_variable('c1_w',[5,5,1,8],dtype=tf.float32)#[5,5,1,8]1输入1，输出8通道使用8个5*5的卷积核每个卷积核有1个通道
    #diyigebianzhizhi
    conv1_bias=tf.get_variable('c1_b',[8],dtype=tf.float32)#8个图分别加一个bias
    
    #第一层卷积
    conv1=tf.nn.conv2d(inputs,conv1_weight,[1,1,1,1],'VALID')
    conv1=tf.nn.relu(conv1+conv1_bias)#激活函数是relu，偏置值bias
    #输出shape=[none,24,24,8] 8片
    
    #池化
    pool1=tf.nn.max_pool(conv1,[1,2,2,1],[1,2,2,1],'SAME')#输入，滑窗，步长，处理方式
    #输出shape=[None,12,12,8]
    
    
    #第二层卷积
    #权重
    conv2_weight=tf.get_variable('c2_w',[5,5,8,16],dtype=tf.float32)#[5,5,8,16]8输入8，输出16通道使用16个5*5的卷积核每个卷积核有8个通道
    #第二层偏置
    conv2_bias=tf.get_variable('c2_b',[16],dtype=tf.float32)#16个图分别加一个bias
    
    #第二层卷积
    conv2=tf.nn.conv2d(pool1,conv2_weight,[1,1,1,1],'VALID')
    conv2=tf.nn.relu(conv2+conv2_bias)#激活函数是relu，偏置值bias
    #输出【none,8,8,16】
    
    #池化
    pool2=tf.nn.max_pool(conv2,[1,2,2,1],[1,2,2,1],'SAME')#输入，滑窗，步长，处理方式
    #输出shape=[none,4,4,16]
    
    
    #为了衔接全连接输出，把shape变成二维
    tmp=tf.reshape(pool2,[-1,4*4*16])
    #或者
   # pool2_shape=tf.shape(pool2)
    #tmp=tf.reshape(pool2,pool2_shape[1]*pool2_shape[2]*pool2_shape[3])
    
    #全连接层
    logits = tf.keras.layers.Dense(10, activation=None)(tmp)
    output=tf.nn.softmax(logits)
    
    
    saver = tf.train.Saver()
    
    #代价函数   
    loss = tf.reduce_mean(-1 * tf.reduce_sum(
        labels * tf.log(output + 1e-7),
        axis=1))
     #正确率衡量准确性
    acc=tf.reduce_mean(
        tf.cast(
        tf.equal( tf.argmax(output,axis=1) ,tf.argmax(labels,axis=1) )  #比较输出和标记是否相等
        ,tf.float32)#把返回值转为float32类型
    )#reduce_mean会把所有的都加起来求和
    

In [15]:
#tf.nn.conv2d??

In [16]:
##tf.nn.avg_pool??

In [18]:
with tf.Session(graph=g) as sess:
    # 定义梯度下降法优化器
    optim = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    train_op = optim.minimize(loss)
    sess.run(tf.global_variables_initializer())
    
    # 训练模型
    for step in range(20000):
        batch_images, batch_labels = mnist.train.next_batch(32)
        res_loss, _ = sess.run([loss, train_op], feed_dict={
            inputs: batch_images,
            labels: batch_labels
        })
        
        # 输出代价并验证模型
        if step % 500 == 0:
            accs = []
            for test_step in range(10000 // 32):
                batch_images, batch_labels = mnist.test.next_batch(32)
                res_acc = sess.run(acc, feed_dict={
                    inputs: batch_images,
                    labels: batch_labels
                })
                accs.append(res_acc)
            accs = np.mean(accs)
            print('step %5d, loss %2.4f, acc %.4f' % (step, res_loss, accs))
           # saver.save(sess, 'conv_ckpt')   

step     0, loss 2.2614, acc 0.1634
step   500, loss 0.4207, acc 0.8767
step  1000, loss 0.1884, acc 0.9169
step  1500, loss 0.1516, acc 0.9320
step  2000, loss 0.1683, acc 0.9478
step  2500, loss 0.1531, acc 0.9513
step  3000, loss 0.1095, acc 0.9607
step  3500, loss 0.0324, acc 0.9604
step  4000, loss 0.0152, acc 0.9621
step  4500, loss 0.2760, acc 0.9690
step  5000, loss 0.0140, acc 0.9704
step  5500, loss 0.0545, acc 0.9704
step  6000, loss 0.0421, acc 0.9706
step  6500, loss 0.0257, acc 0.9731
step  7000, loss 0.0247, acc 0.9743
step  7500, loss 0.0941, acc 0.9752
step  8000, loss 0.0485, acc 0.9775
step  8500, loss 0.0336, acc 0.9757
step  9000, loss 0.0631, acc 0.9763
step  9500, loss 0.0725, acc 0.9752
step 10000, loss 0.0546, acc 0.9787
step 10500, loss 0.0515, acc 0.9771
step 11000, loss 0.0663, acc 0.9787
step 11500, loss 0.0761, acc 0.9779
step 12000, loss 0.1183, acc 0.9799
step 12500, loss 0.0108, acc 0.9797
step 13000, loss 0.0861, acc 0.9802
step 13500, loss 0.0054, acc

In [19]:
tf.keras.layers.Dense??